# PROJECT 1: Simple navigation using DQN

In [1]:
import os
import sys
import numpy as np

from navigation.envs import mlagents

In [2]:
# definitions for full paths to executables
EXEC_BANANA_PATH = os.path.join( os.getcwd(), 'executables/Banana_Linux/Banana.x86_64' )
EXEC_BANANA_NOVIS_PATH = os.path.join( os.getcwd(), 'executables/Banana_Linux_NoVis/Banana.x86_64' )

In [3]:
# initialize the environment
_env = mlagents.createDiscreteActionsEnv( EXEC_BANANA_PATH )
## _env = mlagents.createDiscreteActionsEnv( EXEC_BANANA_NOVIS_PATH )

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


In [4]:
print( 'numActions: ', _env.numActions )
print( 'obsShape: ', _env.obsShape )

numActions:  4
obsShape:  (37,)


In [29]:
_state = _env.reset( training = False )
print( '_state.shape: ', _state.shape )
print( '_state:' )
_raysState = _state[:-2].reshape( 7, -1 )
print( '_raysState: ' )
print( _raysState )

def process( state ) :
    return state[:-2].reshape( 7, -1 )

_state.shape:  (37,)
_state:
_raysState: 
[[0.         0.         1.         0.         0.34546468]
 [1.         0.         0.         0.         0.31790283]
 [1.         0.         0.         0.         0.10271699]
 [1.         0.         0.         0.         0.42798778]
 [1.         0.         0.         0.         0.40745682]
 [1.         0.         0.         0.         0.33010077]
 [1.         0.         0.         0.         0.26187932]]


In [28]:
_raysState[:,[0,2,4]]

array([[1.        , 0.        , 0.24892281],
       [0.        , 1.        , 0.54682684],
       [0.        , 0.        , 0.26143727],
       [1.        , 0.        , 0.38079441],
       [0.        , 0.        , 0.45378795],
       [1.        , 0.        , 0.18091735],
       [0.        , 1.        , 0.27510968]])

In [30]:
_state = _env.reset( training = False )

while True :
    
    _action = np.random.randint( _env.numActions )
    _snext, _reward, _done, _ = _env.step( 2 )
    
    print( '#######################' )
    print( '_snext: ' )
    print( process( _snext ) )
    print( '_reward: ', _reward )
    print( '_done: ', _done )
    
    if _done :
        break



#######################
_snext: 
[[1.         0.         0.         0.         0.23939241]
 [0.         1.         0.         0.         0.75913167]
 [1.         0.         0.         0.         0.02912213]
 [0.         0.         1.         0.         0.50381249]
 [1.         0.         0.         0.         0.10443324]
 [0.         0.         0.         1.         0.        ]
 [1.         0.         0.         0.         0.09014834]]
_reward:  0.0
_done:  False
#######################
_snext: 
[[1.         0.         0.         0.         0.69015008]
 [1.         0.         0.         0.         0.07960904]
 [1.         0.         0.         0.         0.04516501]
 [0.         0.         1.         0.         0.26781228]
 [1.         0.         0.         0.         0.02788808]
 [1.         0.         0.         0.         0.10323893]
 [1.         0.         0.         0.         0.05666176]]
_reward:  0.0
_done:  False
#######################
_snext: 
[[0.         0.         1.     

#######################
_snext: 
[[0.         1.         0.         0.         0.5731113 ]
 [0.         0.         0.         1.         0.        ]
 [0.         0.         1.         0.         0.62675673]
 [0.         1.         0.         0.         0.58886439]
 [1.         0.         0.         0.         0.5182901 ]
 [0.         0.         1.         0.         0.52621138]
 [1.         0.         0.         0.         0.65606463]]
_reward:  0.0
_done:  False
#######################
_snext: 
[[0.         1.         0.         0.         0.60632396]
 [0.         0.         1.         0.         0.25344363]
 [0.         0.         0.         1.         0.        ]
 [0.         1.         0.         0.         0.8148585 ]
 [0.         0.         0.         1.         0.        ]
 [0.         0.         0.         1.         0.        ]
 [1.         0.         0.         0.         0.55871183]]
_reward:  0.0
_done:  False
#######################
_snext: 
[[0.         1.         0.     

 [0.         0.         1.         0.         0.26185566]]
_reward:  0.0
_done:  False
#######################
_snext: 
[[1.         0.         0.         0.         0.2317103 ]
 [0.         1.         0.         0.         0.75913996]
 [1.         0.         0.         0.         0.02691747]
 [0.         0.         1.         0.         0.50135303]
 [1.         0.         0.         0.         0.0932217 ]
 [0.         0.         1.         0.         0.41638947]
 [1.         0.         0.         0.         0.08744544]]
_reward:  0.0
_done:  False
#######################
_snext: 
[[1.         0.         0.         0.         0.68927324]
 [1.         0.         0.         0.         0.07863221]
 [1.         0.         0.         0.         0.04333578]
 [0.         0.         1.         0.         0.26690647]
 [1.         0.         0.         0.         0.02699278]
 [1.         0.         0.         0.         0.10197326]
 [1.         0.         0.         0.         0.05220382]]
_rewa

 [1.         0.         0.         0.         0.55869526]]
_reward:  0.0
_done:  False
#######################
_snext: 
[[0.         1.         0.         0.         0.89943522]
 [1.         0.         0.         0.         0.52688318]
 [0.         0.         1.         0.         0.38107258]
 [0.         0.         0.         1.         0.        ]
 [0.         0.         0.         1.         0.        ]
 [1.         0.         0.         0.         0.78196353]
 [0.         0.         1.         0.         0.99616086]]
_reward:  0.0
_done:  False
#######################
_snext: 
[[1.         0.         0.         0.         0.65607268]
 [0.         0.         0.         1.         0.        ]
 [0.         0.         1.         0.         0.41638172]
 [1.         0.         0.         0.         0.51828402]
 [0.         0.         1.         0.         0.50135005]
 [0.         0.         1.         0.         0.626755  ]
 [1.         0.         0.         0.         0.23171917]]
_rewa

 [1.         0.         0.         0.         0.0279105 ]]
_reward:  0.0
_done:  False
#######################
_snext: 
[[1.         0.         0.         0.         0.08744672]
 [1.         0.         0.         0.         0.03327365]
 [0.         0.         1.         0.         0.52619624]
 [1.         0.         0.         0.         0.09321226]
 [0.         1.         0.         0.         0.58885932]
 [1.         0.         0.         0.         0.02691754]
 [0.         1.         0.         0.         0.57311398]]
_reward:  0.0
_done:  False
#######################
_snext: 
[[1.         0.         0.         0.         0.05220912]
 [0.         1.         0.         0.         0.56614023]
 [0.         0.         0.         1.         0.        ]
 [1.         0.         0.         0.         0.02699267]
 [0.         1.         0.         0.         0.81483299]
 [1.         0.         0.         0.         0.04333416]
 [0.         1.         0.         0.         0.60631692]]
_rewa

...

#######################
_snext: 
[[0.         0.         1.         0.         0.26185986]
 [1.         0.         0.         0.         0.02963464]
 [0.         1.         0.         0.         0.57663649]
 [1.         0.         0.         0.         0.09380154]
 [0.         0.         1.         0.         0.20468454]
 [1.         0.         0.         0.         0.07627323]
 [1.         0.         0.         0.         0.02790923]]
_reward:  0.0
_done:  False
#######################
_snext: 
[[1.         0.         0.         0.         0.0874514 ]
 [1.         0.         0.         0.         0.03327052]
 [0.         0.         1.         0.         0.52615851]
 [1.         0.         0.         0.         0.09317758]
 [0.         1.         0.         0.         0.58884686]
 [1.         0.         0.         0.         0.02691777]
 [0.         1.         0.         0.         0.57312071]]
_reward:  0.0
_done:  False
#######################
_snext: 
[[1.         0.         0.     

 [0.         0.         1.         0.         0.99465734]]
_reward:  0.0
_done:  False
#######################
_snext: 
[[1.         0.         0.         0.         0.65610796]
 [0.         0.         0.         1.         0.        ]
 [0.         0.         1.         0.         0.4163188 ]
 [1.         0.         0.         0.         0.51825774]
 [0.         0.         1.         0.         0.50132632]
 [0.         0.         1.         0.         0.62674773]
 [1.         0.         0.         0.         0.23179123]]
_reward:  0.0
_done:  False
#######################
_snext: 
[[1.         0.         0.         0.         0.55861628]
 [1.         0.         0.         0.         0.21568316]
 [1.         0.         0.         0.         0.10198417]
 [0.         0.         0.         1.         0.        ]
 [0.         0.         1.         0.         0.26691377]
 [0.         0.         0.         1.         0.        ]
 [1.         0.         0.         0.         0.6892755 ]]
_rewa

#######################
_snext: 
[[1.         0.         0.         0.         0.02790888]
 [0.         0.         1.         0.         0.27815202]
 [1.         0.         0.         0.         0.78161806]
 [0.         0.         1.         0.         0.2046763 ]
 [0.         0.         0.         1.         0.        ]
 [0.         1.         0.         0.         0.57663405]
 [0.         1.         0.         0.         0.89932054]]
_reward:  0.0
_done:  False
#######################
_snext: 
[[0.         1.         0.         0.         0.57312262]
 [0.         0.         0.         1.         0.        ]
 [0.         0.         1.         0.         0.62674677]
 [0.         1.         0.         0.         0.58884346]
 [1.         0.         0.         0.         0.51825464]
 [0.         0.         1.         0.         0.5261479 ]
 [1.         0.         0.         0.         0.65611213]]
_reward:  0.0
_done:  False
#######################
_snext: 
[[0.         1.         0.     

 [1.         0.         0.         0.         0.08745333]]
_reward:  0.0
_done:  False
#######################
_snext: 
[[1.         0.         0.         0.         0.68927598]
 [1.         0.         0.         0.         0.07863697]
 [1.         0.         0.         0.         0.04332588]
 [0.         0.         1.         0.         0.2669152 ]
 [1.         0.         0.         0.         0.02699209]
 [1.         0.         0.         0.         0.10198637]
 [1.         0.         0.         0.         0.05223615]]
_reward:  0.0
_done:  False
#######################
_snext: 
[[0.         0.         1.         0.         0.26186135]
 [1.         0.         0.         0.         0.02963577]
 [0.         1.         0.         0.         0.57663286]
 [1.         0.         0.         0.         0.09380416]
 [0.         0.         1.         0.         0.20467199]
 [1.         0.         0.         0.         0.07627199]
 [1.         0.         0.         0.         0.0279087 ]]
_rewa

 [0.         0.         1.         0.         0.99414736]]
_reward:  0.0
_done:  False
#######################
_snext: 
[[1.         0.         0.         0.         0.65612   ]
 [0.         0.         0.         1.         0.        ]
 [0.         0.         1.         0.         0.41629767]
 [1.         0.         0.         0.         0.51824892]
 [0.         0.         1.         0.         0.50131828]
 [0.         0.         1.         0.         0.62674522]
 [1.         0.         0.         0.         0.23181555]]
_reward:  0.0
_done:  False
#######################
_snext: 
[[1.         0.         0.         0.         0.55859059]
 [1.         0.         0.         0.         0.21567979]
 [1.         0.         0.         0.         0.10198747]
 [0.         0.         0.         1.         0.        ]
 [0.         0.         1.         0.         0.26691598]
 [0.         0.         0.         1.         0.        ]
 [1.         0.         0.         0.         0.6892761 ]]
_rewa